# PhosphoELM Data Formating

This file takes data regarding kinase-protein interactions from the PhosphoELM database and converts the data into the .gmt format. The data was retrieved from the PhosphoELM database on Wed, Jun 7 2017 16:27:31. This data will be added to enhance the KEA2 database and will be suitably formatted for use by ENRICHR and X2K.

In [ ]:
import numpy as np
import pandas as pd
import xmltodict
import json
import requests

In [ ]:
#read data from excel file into dataframe 'phospho_df'
phospho_df = pd.read_excel('~/Desktop/phosphoELM_all_2015-04.xlsm')

# This is a title
## This is a subtitle

hello you can write in **bold**, *italic*

lists:
- asd
- asd
- asd


numbered lists:
1. asd
2. 234
3. 46

In [ ]:
#select columns necessary for .gmt format and filter into new dataframe 'df'
df = phospho_df[['acc', 'kinases']]

#drop all columns with an 'NaN' value for the kinases
df.dropna(axis = 0, inplace = True)

#drop duplicate rows in the dataframe
df.drop_duplicates(inplace = True)

#set index of protein values 'acc' as kinases
#creates new dataframe 'kin'
kin = df.set_index('kinases')

#Create dictionary 'PhosphoELM' with kinases as keys
PhosphoELM = dict([(key, []) for key in kin.index])

In [ ]:
#Define url to obtain gene symbol from API
ENRICHR_URL = 'https://www.ebi.ac.uk/proteins/api/proteins/%s'

#Define function uniprot_to_gene which converts uniprot_id into the gene symbol
def uniprot_to_gene(protein_id):
    response = requests.get(ENRICHR_URL % protein_id)
    if not response.ok:
        name = np.NaN
    else:
        data = xmltodict.parse(response.text)
        entry = data['entry']
        # check if entry contains 'gene'
        if 'gene' in entry.keys():
            data = data['entry']['gene']
            if type(data) == list:
                name = str(protein_id)
            else:
                names = data['name']
                if type(names) == list:
                    name = list(names[0].values())[1]
                else:
                    name = list(names.values())[1]
        else:
            name = data['entry']['name']         
    return name

    
for key, row in kin.acc.iteritems():
    
    protein_id = '%s' %row
    geneS = uniprot_to_gene(protein_id)
    PhosphoELM[key] = PhosphoELM[key] + [geneS]
    

kin = pd.DaatFrame.from_dict(PhosphoELM, orient = index)

kin.head()

In [ ]:
#Group kinases in dataframe 'kin'
#Aggregate data in 'kin' according to kinase groups
kin = kin.groupby('kinases').agg(lambda x: tuple(x))

#Create a new column 'PhosphoELM' as description of data
kin.insert(0, 'Description', 'PhosphoELM')

In [ ]:
### NOTES to self

#look into plotly

In [ ]:
# fix the dataframe in order to have three columns:
# kinases, description, acc_merged (acc, but all elements are joined by a \t symbol)
# with a reset index

#reset index of the datframe to integers, restores column 'kinases'
kin.reset_index(inplace = True)

#create column 'acc_merged' in which all 'acc' elements are joined by a \t symbol
kin['acc_merged'] = ['\t'.join(x) for x in kin['acc']]

acc = pd.Series(kin['acc'])

acc[4]

#drop the now-unneccesary column 'acc'
kin.drop('acc', axis=1, inplace = True)

#Create dictionary 'PhosphoELM' with kinases as keys
PhosphoELM = dict([(key, []) for key in kin.index])

# loop through rows with iterrows()
for index, rowData in kin.iterrows():
    line = ['\t'.join(rowData)]
    PhosphoELM[index] = line

In [ ]:
#Transfer tsv info into a new txt file
with open('PhosphoELM.txt', 'w') as openfile:
    for index in PhosphoELM:
        openfile.write(str(PhosphoELM[index]) + '\n')